# Quality assurance and quality control

## WAFS

In [1]:
import glob
import os
import sys

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import numpy as np
import pandas as pd
import xarray as xr
from metpy.units import units

sys.path.insert(1, "/Users/lukeconibear/repos/wxapps/")

from apps.wafs.src.wafs import utils, wafs

plt.rcParams.update(
    {
        "text.usetex": True,
        "font.family": "serif",
        "font.sans-serif": ["Computer Modern Roman"],
        "axes.grid": False,
        "savefig.dpi": 700,
        "figure.figsize": [8, 6],
        "font.size": 14,
    }
)
plt.style.use(["seaborn-colorblind"])
xr.set_options(display_expand_data=False)

not handlers
not handlers
not handlers


- for each forecast time
    - have raw files and converted files
    - regrid raw files to the same grid
    - ensure units are the same
    - for each variable in wind speed, wind direction, and air temperature
        - for each height
            - diff the files
            - summarise the diff (should be small)

In [18]:
batch_timestamps = pd.date_range(start="2022-05-02", end="2022-06-30", freq="6H")
variables = ["air_temperature_aviation", "wind_speed_aviation", "wind_direction_aviation"]

In [3]:
for batch_timestamp in batch_timestamps[0:1]:
    forecast_times = [batch_timestamp + pd.to_timedelta(hour) for hour in ['6H', '9H']]
    for forecast_time in forecast_times:
        print(forecast_time)

2022-05-02 06:00:00
2022-05-02 09:00:00


In [4]:
def evaluate_forecast(forecast_time):
    filename = f"""
        ingress_met_model_WAF_{batch_timestamp.year}_
        {batch_timestamp.month:02}_{batch_timestamp.day:02}_
        KWBC_{batch_timestamp.hour:02}_Wind_temp_{batch_timestamp.year}
        {batch_timestamp.month:02}{batch_timestamp.day:02}_
        {batch_timestamp.hour:02}00f{forecast_time.hour:02}.grib2
    """.replace('\n', '').replace(' ', '')

    return filename

In [5]:
filename = evaluate_forecast(forecast_time)
filename

'ingress_met_model_WAF_2022_05_02_KWBC_00_Wind_temp_20220502_0000f09.grib2'

In [6]:
ds_raw_grib = wafs.open_grib_dataset(
    filename,
    height_dimension="isobaricInhPa",
)
u_wind_component = ds_raw_grib["u"] * units("m/s")
v_wind_component = ds_raw_grib["v"] * units("m/s")
wind_speed_aviation = mpcalc.wind_speed(u_wind_component, v_wind_component)
wind_direction_aviation = mpcalc.wind_direction(u_wind_component, v_wind_component)
air_temperature_aviation = ds_raw_grib["t"].metpy.convert_units("degC")

ds_wind_temp = xr.combine_by_coords(
    [
        xr.Dataset({"air_temperature_aviation": air_temperature_aviation}),
        xr.Dataset({"wind_speed_aviation": wind_speed_aviation}),
        xr.Dataset({"wind_direction_aviation": wind_direction_aviation}),
    ]
)
ds_wind_temp_converted = wafs.convert_pressure_to_height(
    ds_wind_temp,
    old_height_dimension="isobaricInhPa",
    new_height_dimension="lev",
    old_units="hPa",
    new_units="ft",
)
flight_altitudes = list(range(3_000, 56_000, 1_000))
ds_wind_temp_interpolated = wafs.interpolate_to_flight_altitudes(
    ds_wind_temp_converted,
    flight_altitudes,
)

ds_wind_temp_regridded = wafs.regrid_ds(
    ds_wind_temp_interpolated,
    regridding_weights_filename="bilinear_145x288_721x1440_peri.nc",
)

skipping variable: paramId==157 shortName='r'
Traceback (most recent call last):
  File "/Users/lukeconibear/miniconda3/envs/pangeo_intel/lib/python3.9/site-packages/cfgrib/dataset.py", line 660, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/Users/lukeconibear/miniconda3/envs/pangeo_intel/lib/python3.9/site-packages/cfgrib/dataset.py", line 591, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='isobaricInhPa' value=Variable(dimensions=('isobaricInhPa',), data=array([850., 750., 700., 600., 500., 450., 400., 350., 300., 275., 250.,
       225., 200., 175., 150., 125., 100.])) new_value=Variable(dimensions=('isobaricInhPa',), data=array([850., 750., 700., 600., 500.]))


In [7]:
ds_wind_temp = ds_wind_temp.metpy.dequantify()
ds_wind_temp

<xarray.Dataset>
Dimensions:                   (isobaricInhPa: 17, latitude: 145, longitude: 288)
Coordinates:
    time                      datetime64[ns] 2022-05-02
    step                      timedelta64[ns] 09:00:00
  * isobaricInhPa             (isobaricInhPa) float64 850.0 750.0 ... 100.0
  * latitude                  (latitude) float64 90.0 88.75 ... -88.75 -90.0
  * longitude                 (longitude) float64 0.0 1.25 2.5 ... 357.5 358.8
    valid_time                datetime64[ns] 2022-05-02T09:00:00
Data variables:
    air_temperature_aviation  (isobaricInhPa, latitude, longitude) float32 -1...
    wind_direction_aviation   (isobaricInhPa, latitude, longitude) float32 35...
    wind_speed_aviation       (isobaricInhPa, latitude, longitude) float32 11...

In [8]:
ds_wind_temp_regridded

<xarray.Dataset>
Dimensions:                   (lev: 53, lat: 721, lon: 1440)
Coordinates:
    time                      datetime64[ns] 2022-05-02
    step                      timedelta64[ns] 09:00:00
    valid_time                datetime64[ns] 2022-05-02T09:00:00
  * lev                       (lev) int64 3000 4000 5000 ... 53000 54000 55000
  * lat                       (lat) float64 -90.0 -89.75 -89.5 ... 89.75 90.0
  * lon                       (lon) float64 -180.0 -179.8 -179.5 ... 179.5 179.8
Data variables:
    air_temperature_aviation  (lev, lat, lon) float64 -31.45 -31.45 ... -47.79
    wind_direction_aviation   (lev, lat, lon) float64 344.7 344.7 ... 90.61
    wind_speed_aviation       (lev, lat, lon) float64 3.991 3.991 ... 6.533
Attributes:
    regrid_method:  bilinear

In [9]:
approximate_altitude_hpa_vs_feet = {}
for pressure_hpa in ds_wind_temp.isobaricInhPa.values:
    height_pressure_feet_rounded = round(
        mpcalc.pressure_to_height_std(pressure_hpa * units("hPa"))
        .to(units("ft"))
        .magnitude,
        -3,
    )
    approximate_altitude_hpa_vs_feet[str(int(pressure_hpa))] = str(
        int(height_pressure_feet_rounded)
    )

In [10]:
indices_hpa = {}
for pressure_hpa in ds_wind_temp.isobaricInhPa.values:
    index_hpa = np.argwhere(
        ds_wind_temp.isobaricInhPa.values == pressure_hpa
    )
    indices_hpa[str(int(pressure_hpa))] = str(int(index_hpa))

In [11]:
indices_feet = {}
for (
    altitude_feet
) in ds_wind_temp_regridded.lev.values:
    index_feet = np.argwhere(
        ds_wind_temp_regridded.lev.values
        == altitude_feet
    )
    indices_feet[str(int(altitude_feet))] = str(int(index_feet))

In [5]:
ratios = [round(ratio, 1) for ratio in np.linspace(0.1, 0.5, 5)]
percentages_of_ratios_within = {str(ratio): [] for ratio in ratios}
diffs = [int(diff) for diff in np.linspace(1, 5, 5)]
percentages_of_diffs_within = {str(diff): [] for diff in diffs}

In [13]:
for (
    altitude_hpa,
    altitude_feet,
) in approximate_altitude_hpa_vs_feet.items():
    if (
        float(altitude_feet) < 36089
    ):  # the barometric conversion formula only holds for this
        index_hpa = int(indices_hpa[altitude_hpa])
        index_feet = int(indices_feet[altitude_feet])
        print(index_hpa, index_feet)

0 2
1 5
2 7
3 11
4 15
5 18
6 21
7 24
8 27
9 29
10 31
11 33


In [19]:
variable = variables[0]

In [ ]:
for variable in variables:
    




## the memory for this step blows up
## do on gcp



In [ ]:
actual_diff = ds_wind_temp[variable].isel(
    isobaricInhPa=index_hpa
) - ds_wind_temp_regridded[variable].isel(
    lev=index_feet
)
actual_ratio = ds_wind_temp[variable].isel(
    isobaricInhPa=index_hpa
) / ds_wind_temp_regridded[variable].isel(
    lev=index_feet
)

In [6]:
ratios

[0.1, 0.2, 0.3, 0.4, 0.5]

In [ ]:
for ratio in ratios:
    actual_ratio_within = actual_ratio.where(
        cond=np.abs(actual_ratio) > (1.0 - ratio)
    ).where(cond=np.abs(actual_ratio) < (1.0 + ratio))

    percentage_of_values_within = 1 - (
        actual_ratio_within.isnull().sum().values
        / (
            actual_ratio_within.lon.shape[0]
            * actual_ratio_within.lat.shape[0]
        )
    )

    percentages_of_ratios_within[str(ratio)].append(
        percentage_of_values_within
    )

for diff in diffs:
    actual_diff_within = actual_diff.where(
        cond=np.abs(actual_diff) < diff
    )

    percentage_of_diffs_within = 1 - (
        actual_diff_within.isnull().sum().values
        / (
            actual_diff_within.lon.shape[0]
            * actual_diff_within.lat.shape[0]
        )
    )

    percentages_of_diffs_within[str(diff)].append(
        percentage_of_diffs_within
    )

In [ ]:
for ratio, values in percentages_of_ratios_within.items():
    print(
        f"Percentage of ratios within {100 * float(ratio):0.0f}%: {100 * np.mean(values):0.0f}%"
    )

for diff, values in percentages_of_diffs_within.items():
    print(
        f"Percentage of diffs within {float(diff):0.1f} degrees C: {100 * np.mean(values):0.0f}%"
    )

In [ ]:
percentages_of_ratios_within

In [ ]:
percentages_of_diffs_within

In [ ]:
if assert_criteria_to_thresholds:
    assert np.mean(percentages_of_ratios_within["0.1"]) > 0.6
    assert np.mean(percentages_of_ratios_within["0.2"]) > 0.85
    assert np.mean(percentages_of_ratios_within["0.3"]) > 0.9
    assert np.mean(percentages_of_ratios_within["0.4"]) > 0.9
    assert np.mean(percentages_of_ratios_within["0.5"]) > 0.95

    assert np.mean(percentages_of_diffs_within["1"]) > 0.4
    assert np.mean(percentages_of_diffs_within["2"]) > 0.6
    assert np.mean(percentages_of_diffs_within["3"]) > 0.7
    assert np.mean(percentages_of_diffs_within["4"]) > 0.8
    assert np.mean(percentages_of_diffs_within["5"]) > 0.9

### Single datasets

In [ ]:
ds_wind_temp = xr.open_dataset(
    "../../pangeo_tools/wafs/wafs_[wind_temp]_20220630T120000Z_[20220630T210000Z].nc"
)
ds_ice = xr.open_dataset(
    "../../pangeo_tools/wafs/wafs_[ice]_20220630T120000Z_[20220630T210000Z].nc"
)
ds_turb = xr.open_dataset(
    "../../pangeo_tools/wafs/wafs_[turb]_20220630T120000Z_[20220630T210000Z].nc"
)

In [ ]:
ds_wind_temp

In [ ]:
ds_wind_temp.lat

In [ ]:
ds_ice.lat

In [ ]:
ds_turb.lat

In [ ]:
ds_wind_temp.lon

In [ ]:
ds_ice.lon

In [ ]:
ds_turb.lon

In [ ]:
ds_wind_temp.lev

In [ ]:
ds_ice.lev

In [ ]:
ds_turb.lev

In [ ]:
ds_wind_temp.air_temperature_aviation

In [ ]:
ds_wind_temp.air_temperature_aviation.values[0]

In [ ]:
ds_wind_temp.wind_speed_aviation

In [ ]:
ds_wind_temp.wind_speed_aviation.values[0]

In [ ]:
ds_wind_temp.wind_direction_aviation

In [ ]:
ds_wind_temp.wind_direction_aviation.values[0]

In [ ]:
ds_ice.icing_severity_index_aviation

In [ ]:
ds_turb.eddy_dissipation_rate_aviation

In [ ]:
ds_turb.eddy_dissipation_rate_aviation.values[0]

In [ ]:
ds_wind_temp["air_temperature_aviation"].sel(lev=10000).plot();

In [ ]:
ds_wind_temp["air_temperature_aviation"].interp(
    lev=10501, kwargs={"fill_value": "extrapolate"}
).plot();

In [ ]:
colors = {
    "icing_severity_index_aviation": [
        "#e06666",
        "#ffffff",
        "#eff3ff",
        "#bdd7e7",
        "#6baed6",
        "#2171b5",
    ],
    "eddy_dissipation_rate_aviation": [
        "#ffffff",
        "#ffffcc",
        "#ffeda0",
        "#fed976",
        "#feb24c",
        "#fd8d3c",
        "#fc4e2a",
        "#e31a1c",
        "#bd0026",
        "#800026",
    ],
}
levels = {
    "icing_severity_index_aviation": range(-1, 6),
    "eddy_dissipation_rate_aviation": np.linspace(0, 1.0, 11),
}

In [ ]:
variable = "icing_severity_index_aviation"

In [ ]:
ds_ice.lev.values

In [ ]:
ds_ice[variable].sel(lev=10000).plot(levels=levels[variable], colors=colors[variable]);

In [ ]:
ds_ice[variable].interp(lev=10500, kwargs={"fill_value": "extrapolate"}).plot(
    levels=levels[variable], colors=colors[variable]
);

In [ ]:
ds_ice[variable].interp(lev=11000, kwargs={"fill_value": "extrapolate"}).plot(
    levels=levels[variable], colors=colors[variable]
);

In [ ]:
variable = "eddy_dissipation_rate_aviation"

In [ ]:
ds_turb.lev.values

In [ ]:
ds_turb[variable].sel(lev=10000).plot(levels=levels[variable], colors=colors[variable]);

In [ ]:
ds_turb[variable].interp(lev=10500, kwargs={"fill_value": "extrapolate"}).plot(
    levels=levels[variable], colors=colors[variable]
);

In [ ]:
ds_turb[variable].interp(lev=11000, kwargs={"fill_value": "extrapolate"}).plot(
    levels=levels[variable], colors=colors[variable]
);

In [ ]:
variables = set(ds_wind_temp.data_vars) | set(ds_ice.data_vars) | set(ds_turb.data_vars)

In [ ]:
fig = plt.figure(1, figsize=(13, 10))
gs = gridspec.GridSpec(3, 2)

for index, variable in enumerate(variables):
    ax = fig.add_subplot(gs[index])
    if variable == "icing_severity_index_aviation":
        ds_ice[variable].sel(lev=14000).plot(
            ax=ax, levels=levels[variable], colors=colors[variable]
        )
    elif variable == "eddy_dissipation_rate_aviation":
        ds_turb[variable].sel(lev=14000).plot(
            ax=ax, levels=levels[variable], colors=colors[variable]
        )
    else:
        ds_wind_temp[variable].sel(lev=14000).plot(ax=ax)

plt.tight_layout()
# plt.savefig('wafs_20220630T150000Z.png', facecolor="white", dpi=300, bbox_inches="tight")
plt.show()

### Merged datasets

In [ ]:
wafs_files = glob.glob("../../pangeo_tools/wafs/wafs_*20220630T*.nc")

In [ ]:
wafs_files

In [ ]:
datasets = {}

for variable in ["wind_temp", "ice", "turb"]:
    wafs_files_variable = sorted([file for file in wafs_files if variable in file])
    datasets_variable = [xr.open_dataset(file) for file in wafs_files_variable]
    ds_combined_variable = xr.combine_by_coords(
        datasets_variable, combine_attrs="drop_conflicts"
    )
    datasets[variable] = ds_combined_variable

In [ ]:
datasets

In [ ]:
datasets["wind_temp"]["air_temperature_aviation"]

In [ ]:
datasets["wind_temp"]["air_temperature_aviation"].isel(lev=3).plot(
    col="time",
    col_wrap=4,
);

In [ ]:
datasets["ice"]["icing_severity_index_aviation"].isel(lev=3).plot(
    col="time",
    col_wrap=4,
    levels=levels["icing_severity_index_aviation"],
    colors=colors["icing_severity_index_aviation"],
);

In [ ]:
datasets["turb"]["eddy_dissipation_rate_aviation"].isel(lev=3).plot(
    col="time",
    col_wrap=4,
    levels=levels["eddy_dissipation_rate_aviation"],
    colors=colors["eddy_dissipation_rate_aviation"],
);

In [ ]:
import os

os.getcwd()

In [ ]:
from unittest.mock import MagicMock

In [ ]:
mock = MagicMock()
mock.mock_open.side_effect = IOError()
mock.mock_open()

In [ ]:
mock.mock_open.side_effect = IOError